In [1]:
# 基于tnesorflow的VGG实现
# 这里VGG由8个卷积层，3个全连接层构成
# 5个卷积块，前2块使用单卷积层，而后3块使用双卷积层
# 每个卷积层填充为1， 窗口为3 * 3步幅为2， 池化层步幅为2，形状为2 * 2
# 全连接层和AlexNet一样，4096 relu，dropout(0.5), 4096 relu，dropout(0.5), 10 sigmoid
# batch_size的值会影响模型的收敛性，在第一开始尝试100的时候怎么都不收敛，增大到130就可以收敛了
import numpy as np
import tensorflow as tf

In [2]:
# 使用GPU进行训练
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# 建构模型
# VGG 单位块 vgg_block()
def vgg_block(num_convs, num_channels):
    '''
    num_convs: 卷积层数
    num_channels: 输出通道数
    '''
    blk = tf.keras.models.Sequential()
    for _ in range(num_convs):
        blk.add(tf.keras.layers.Conv2D(filters=num_channels, kernel_size=3, padding='same', activation='relu'))
    blk.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
    return blk

# VGG
def VGG(conv_arch):
    '''
    vgg_block 的参数tuple集合
    '''
    net = tf.keras.models.Sequential()
    for (num_convs, num_channels) in conv_arch:
        net.add(vgg_block(num_convs, num_channels))
    net.add(tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=4096, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=10, activation='sigmoid')
    ]))
    return net

In [4]:
# 这里为了简化 把通道数降低
conv_arch=((1, 16), (1, 32), (2, 64), (2, 128), (2, 128))
VGG_net = VGG(conv_arch)

In [5]:
X = tf.random.uniform((1,224,224,1))
for blk in VGG_net.layers:
    X = blk(X)
    print(blk.name, 'output shape:\t', X.shape)

sequential_1 output shape:	 (1, 112, 112, 16)
sequential_2 output shape:	 (1, 56, 56, 32)
sequential_3 output shape:	 (1, 28, 28, 64)
sequential_4 output shape:	 (1, 14, 14, 128)
sequential_5 output shape:	 (1, 7, 7, 128)
sequential_6 output shape:	 (1, 10)


In [6]:
# 数据处理阶段，同AlexNet 使用fashion_mnist 将28 * 28 扩充为244*244
class DataLoad():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32) / 255.0, axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32) / 255.0, axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.train_num, self.test_num = self.train_images.shape[0], self.test_images.shape[0]
    
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, self.train_num, batch_size)
        # 填充数据集变为244*244 tf.image
        train_batch = tf.image.resize_with_pad(self.train_images[index], 244, 244)
        return train_batch, self.train_labels[index]
    
    def get_batch_test(self, batch_size):
        index = np.random.randint(0, self.test_num, batch_size)
        # 填充数据集变为244*244 tf.image
        test_batch = tf.image.resize_with_pad(self.test_images[index], 244, 244)
        return test_batch, self.test_labels[index]

In [7]:
batch_size = 120
data_load  = DataLoad()

In [8]:
# 模型训练阶段
def train_VGG():
    epochs = 5
    num_iter = data_load.train_num // batch_size
    for _ in range(epochs):
        for t in range(num_iter):
            X_batch, Y_batch = data_load.get_batch_train(batch_size)
            VGG_net.fit(X_batch, Y_batch)
            if t % 20 == 0:
                VGG_net.save_weights('VGG_net_weights.h5')
# 优化器
optimizer = tf.keras.optimizers.SGD(learning_rate=0.05, momentum=0.0, nesterov=False)
VGG_net.compile(optimizer=optimizer,
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
train_VGG()

4/4 [==============================] - 0s 44ms/step - loss: 0.1829 - accuracy: 0.9333


In [10]:
X_test, Y_test = data_load.get_batch_test(400)
VGG_net.evaluate(X_test, Y_test)

13/13 [==============================] - 1s 37ms/step - loss: 0.1890 - accuracy: 0.9275


[0.18897861242294312, 0.9275000095367432]